### =============================================================
### 셀 1: 필수 라이브러리 설치 및 환경 변수 설정 (RunPod Jupyter Notebook에서 가장 먼저 실행)
### =============================================================

In [ ]:
# ==============================================================================
# 0. 필수 모듈 임포트 및 환경 설정
# ==============================================================================

# RunPod 환경: runpod/pytorch:2.1.0-py3.10-cuda11.8.0-devel-ubuntu22.04

import os
import sys
import pkg_resources
import gc # 가비지 컬렉션을 위함

# pip 및 torch 업그레이드
print("환경 설정 및 필수 라이브러리 설치를 시작합니다.")
!pip install --upgrade pip torch

# ==============================================================================

# --- 1. 현재 커널의 Python 버전 확인 (참고용) ---
print(f"현재 커널의 Python 버전: {sys.version}")

# --- 2. CUDA 버전 확인 ---
print("\n--- CUDA 버전 확인 중 ---")

# --- 3. PyTorch 및 CUDA 버전 확인 (RunPod 환경에 이미 설치된 것 활용) ---
print("\n--- RunPod 환경의 PyTorch 및 CUDA 버전 확인 중 ---")
try:
    import torch # PyTorch 버전 확인을 위함
    print(f"현재 PyTorch 버전: {torch.__version__}")
    print(f"PyTorch CUDA 사용 가능 여부: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"PyTorch가 인식하는 CUDA 버전: {torch.version.cuda}")
        print(f"현재 GPU 이름: {torch.cuda.get_device_name(0)}")
    else:
        print("PyTorch가 CUDA를 인식하지 못합니다. 문제 발생 가능성이 있습니다.")
except ImportError:
    print("PyTorch가 설치되지 않았거나 임포트할 수 없습니다. 수동 설치가 필요할 수 있습니다.")

# --- 4. pip 캐시 완전 삭제 ---
print("\n--- pip 캐시 삭제 중... ---")
!pip cache purge

# --- 5. 핵심 라이브러리 최신 버전 설치 ---
!pip install -U peft trl transformers bitsandbytes typing_extensions accelerate

# --- 6. NumPy 2.0.2 버전 설치 ---
print("\n--- NumPy 2.0.2 버전 설치 중... ---")
!pip install numpy==2.0.2

# --- 7. ipykernel 설치 ---
print("\n--- ipykernel 설치 중... ---")
!pip install ipykernel

# --- 8. 나머지 필수 라이브러리 설치 (datasets, safetensors, langchain 등) ---
print("\n--- 나머지 필수 라이브러리 및 datasets, safetensors 설치 중... ---")
!pip install \
    datasets \
    safetensors \
    langchain \
    langchain-community \
    langchain-core \
    faiss-cpu

# --- 9. 가비지 컬렉션 및 GPU 캐시 정리 ---
print("\n--- 가비지 컬렉션 및 GPU 캐시 정리 중... ---")
gc.collect()
torch.cuda.empty_cache()
print("\n--- 모든 라이브러리 설치 및 환경 변수 설정 완료! ---")

# --- 10. 설치 확인 (모든 주요 라이브러리 임포트 테스트 강화) ---
print("\n--- bitsandbytes 진단 실행 중... ---")
get_ipython().system('python -m bitsandbytes')
print("\n--- 주요 라이브러리 임포트 테스트 ---")
try:
    installed_numpy_version = pkg_resources.get_distribution("numpy").version
    print(f"\n설치된 numpy 버전: {installed_numpy_version}")

    installed_typing_extensions_version = pkg_resources.get_distribution("typing_extensions").version
    print(f"설치된 typing_extensions 버전: {installed_typing_extensions_version}")

    import torch
    print(f"설치된 PyTorch 버전: {torch.__version__}")
    print(f"PyTorch CUDA 사용 가능 여부: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"PyTorch CUDA 버전: {torch.version.cuda}")

    import transformers
    print(f"transformers 버전: {transformers.__version__}")

    import bitsandbytes as bnb
    print(f"bitsandbytes 버전: {bnb.__version__}")

    import accelerate
    print(f"accelerate 버전: {accelerate.__version__}")

    import peft
    print(f"peft 버전: {peft.__version__}")

    import trl
    print(f"trl 버전: {trl.__version__}")

    import tokenizers
    print(f"tokenizers 버전: {tokenizers.__version__}")

    import safetensors
    # safetensors는 'safetensors'라는 이름으로 설치되므로, pkg_resources.get_distribution 사용
    installed_safetensors_version = pkg_resources.get_distribution("safetensors").version
    print(f"safetensors 버전: {installed_safetensors_version}")

    import datasets
    print(f"datasets 버전: {datasets.__version__}")

    import langchain
    print(f"langchain 버전: {langchain.__version__}")

    import faiss
    print(f"faiss-cpu 버전: {faiss.__version__}")

    print("\n--- 모든 주요 라이브러리 임포트 테스트 성공. ---")
except Exception as e:
    print(f"\n--- 라이브러리 확인 중 오류 발생: {e} ---")

# --- 11. 중요: Jupyter 커널 재시작 권장 ---
print("\n--- 경고: Jupyter 커널을 재시작하는 것을 강력히 권장합니다 (메뉴: Kernel -> Restart Kernel). ---")
print("재시작 후, 이 셀을 건너뛰고 바로 다음 셀(본 코드)을 실행하십시오.")

### ===================================================
### 셀 2: Vector DB 생성 코드 (셀 1 실행 및 커널 재시작 후 실행)
### ===================================================

In [ ]:
# ==============================================================================
# 1. 필수 라이브러리 임포트 및 환경설정
# ==============================================================================

import os, gc, json, torch
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_NAME = "nlpai-lab/KURE-v1"
RAG_DATA_PATH = "./data/09_postprocessed_scripts.jsonl"
VECTORSTORE_PATH = "./vectordb/dais_vectorstore.index"
METADATA_PATH = "./vectordb/dais_vectorstore_meta.jsonl"

c:\Users\Playdata\AppData\Local\anaconda3\envs\dais_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ==============================================================================
# 2. 임베딩 모델 및 RAG 데이터 로드
# ==============================================================================

kure_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

rag_texts, rag_metadatas = [], []
error_count = 0
if os.path.exists(RAG_DATA_PATH):
    with open(RAG_DATA_PATH, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            try:
                rec = json.loads(line)
                rag_texts.append(rec.get("output", ""))
                # output(본문)도 메타데이터에 함께 저장 (검색 결과 매핑 안전)
                rag_metadatas.append({"input": rec.get("input", ""), "output": rec.get("output", "")})
            except Exception as e:
                error_count += 1
                print(f"JSON 파싱 오류 (라인 {i+1}): {e} - {line[:100]}")
    if error_count > 0:
        print(f"총 {error_count}건의 JSON 파싱 오류 발생")
else:
    print(f"경고: RAG 데이터 파일이 존재하지 않습니다: {RAG_DATA_PATH}")

In [3]:
# ==============================================================================
# 3. 벡터 스토어(임베딩 및 L2 정규화 + IndexFlatIP 인덱스) 구축 및 저장
# ==============================================================================

if rag_texts:
    print("임베딩 생성 중...")
    # encode에서 L2 정규화 바로 적용
    embeddings = kure_model.encode(rag_texts, convert_to_numpy=True, normalize_embeddings=True)
    embeddings = embeddings.astype(np.float32)
    print(f"임베딩 shape: {embeddings.shape}")

    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    print(f"IndexFlatIP에 {embeddings.shape[0]}개 벡터 추가 완료.")

    faiss.write_index(index, VECTORSTORE_PATH)
    print(f"FAISS 인덱스를 {VECTORSTORE_PATH}에 저장 완료.")

    with open(METADATA_PATH, "w", encoding="utf-8") as f:
        for meta in rag_metadatas:
            f.write(json.dumps(meta, ensure_ascii=False) + "\n")
    print(f"메타 데이터를 {METADATA_PATH}에 저장 완료.")
else:
    print("벡터 스토어 생성 실패: RAG 문서가 없습니다.")

# 메모리/캐시 정리
gc.collect()
torch.cuda.empty_cache()

임베딩 생성 중...
임베딩 shape: (3697, 1024)
IndexFlatIP에 3697개 벡터 추가 완료.
FAISS 인덱스를 ./dais_vectorstore.index에 저장 완료.
메타 데이터를 ./dais_vectorstore_meta.jsonl에 저장 완료.
